# TOTVS Labs Data Challenge: Preprocessing

Import required modules

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import pickle
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

## Data preparation

Reference:
* customer_code: unique id of a customer;
* branch_id: the branch id where this order was made;
* sales_channel: the sales channel this order was made;
* seller_code: seller that made this order;
* register_date: date of the order;
* total_price: total price of the order (sum of all items);
* order_id: id of this order;
* quantity: quantity of items, given by item_code, were bought;
* item_total_price: total price of items, i.e., quantity* price;
* unit_price: unit price of this item;
* group_code: which group this customer belongs;
* segment_code: segment this client belongs;
* is_churn: True, if we believe the client will not come back. (for a `customer_code` this values is always the same)


Load the given data

In [2]:
original_df = pd.read_json("../data/challenge.json")

In [3]:
df = original_df.copy()

In [4]:
df.shape

(204428, 14)

The dataset consists of 204K samples as 14 features... Checking the type of the entries

In [5]:
df.head()

,branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,register_date,sales_channel,segment_code,seller_code,total_price,unit_price
0,0,143,0,0.0,854,292.91,21804,10,2017-11-10T00:00:00Z,0,0,190,1613.53,25.04
1,0,433,0,0.0,246,287.19,5486,20,2011-05-16T00:00:00Z,1,5,153,11163.69,12.33
2,0,486,0,0.0,1420,184.84,22662,12,2018-01-24T00:00:00Z,0,0,166,6432.12,12.80
3,0,107,0,0.0,1963,189.18,3956,18,2010-07-28T00:00:00Z,1,0,156,831.82,10.51
4,0,768,0,0.0,1786,66.87,4730,5,2010-12-17T00:00:00Z,1,0,218,1736.48,11.82


*register_date* is a date, we will convert it to the corresponding type

In [6]:
pd.to_datetime(df["register_date"]).head()

0   2017-11-10 00:00:00+00:00
1   2011-05-16 00:00:00+00:00
2   2018-01-24 00:00:00+00:00
3   2010-07-28 00:00:00+00:00
4   2010-12-17 00:00:00+00:00
Name: register_date, dtype: datetime64[ns, UTC]

they seem to be just dates with no time... let's check that

In [7]:
dates = pd.to_datetime(df["register_date"])

In [8]:
dates[dates.apply(lambda x: x.time()) != datetime.time(0,0)]

Series([], Name: register_date, dtype: datetime64[ns, UTC])

The series is empty, therefore they are all dates and we can just convert them to dates

In [9]:
df["register_date"] = dates.apply(lambda x: x.date())

We check at the types of the other fields...

In [10]:
dtypes = df.dtypes
dtypes[(dtypes != np.int) & (dtypes != np.float)]

register_date    object
dtype: object

We have numeric data (ints and floats) besides the dates already mentioned

In [11]:
df.describe()

,branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,sales_channel,segment_code,seller_code,total_price,unit_price
count,204428.0,204428.000000,204428.000000,202513.000000,204428.000000,204428.000000,204428.000000,204428.000000,204428.000000,204428.000000,204428.000000,204428.000000,204428.000000
mean,0.0,447.489062,0.307184,0.191168,1437.298981,304.048244,12434.088863,12.332694,1.761182,0.846381,145.543022,5337.072931,47.456664
std,0.0,246.678026,0.714639,0.393222,903.166319,1009.268287,6935.623011,53.299314,2.929541,1.615763,75.197609,21268.855032,145.295694
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.010000,0.010000
25%,0.0,221.000000,0.000000,0.000000,631.000000,79.600000,6497.750000,3.000000,0.000000,0.000000,83.000000,1180.470000,14.670000
50%,0.0,486.000000,0.000000,0.000000,1455.000000,156.150000,12698.000000,6.000000,0.000000,0.000000,156.000000,2035.820000,21.780000
75%,0.0,651.000000,0.000000,0.000000,2048.000000,293.862500,18280.000000,12.000000,2.000000,2.000000,195.000000,3611.070000,46.720000
max,0.0,837.000000,3.000000,1.000000,2980.000000,102951.110000,24617.000000,7000.000000,105.000000,7.000000,289.000000,428976.460000,26280.000000


*branch_id* is constant, we may ignore it

In [12]:
df.drop(columns="branch_id", inplace=True)

From the summary we see that column *is_churn* has missing values...

In [13]:
# TODO: we can see if we can fill them in

We save the results for the next steps....

In [14]:
df.to_pickle("../models/df_preprocessed.pkl")

### Summary

* we removed variable *branch_id* because it was constant
* *is_churn* has missing values
* *register_date* is converted to a date
* the processed df is saved to "../models/df_preprocessed.pkl"